# Serverless Transfer Learning with Cloud ML Engine and Keras

The notebook is based on [article](https://medium.com/google-cloud/serverless-transfer-learning-with-cloud-ml-engine-and-keras-335435f31e15).


## Install required package.

Install cloudmlmagic ahead of running.

In [ ]:
!pip install cloudmlmagic

Load cloudmlmagic extension

In [ ]:
%reload_ext cloudmlmagic

Initialize and setup ML Engine parameters.
Following dict will be written in setup.py of your package,
so list up neccesary packages of your code.

** NOTE: Please replace with your own project id and bucket where to save trained model in following two cells **

In [ ]:
%%ml_init -projectId datalab-trial-186403 -bucket cloudml_models/transfer_learning -scaleTier BASIC -region us-central1 -runtimeVersion 1.2 
{'install_requires': ['keras', 'h5py', 'Pillow']}

In [ ]:
%env PROJECTID=datalab-trial-186403
%env BUCKET=cloudml_models/transfer_learning

## Inception-v3 in Keras

It is quite easy to use a pre-trained model in Keras, only two lines as follows.

This model was pre-trained with ImageNet’s datasets, which has one million images and 1000 classes.

![](https://cdn-images-1.medium.com/max/1600/1*ZCXqy5c-MwRzJlo7rYPyRQ.png)

In [ ]:
%%ml_code

from keras.applications.inception_v3 import InceptionV3
model = InceptionV3(weights='imagenet')

In [ ]:
%%ml_code

# Imports Keras packages and model
import numpy as np
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input, decode_predictions

Let’s classify following two images with this model. Since Inception-v3 model accepts RGB 299x299 image as input, you must convert your image before classify it.

In [ ]:
# Inline display image
from IPython.display import display

def predict(img_path):
    img = image.load_img(img_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Display image
    display(img)
    
    # Classify image
    preds = model.predict(x)
    
    # Print predicted classes
    print('Predicted:')
    for p in decode_predictions(preds, top=5)[0]:
        print("Score {}, Label {}".format(p[2], p[1]))    
        
predict('Indian_elephant.jpeg')
predict('Gull.jpeg')        

The model classified elephants correctly, but failed to classify a seagull. The reason is simply because datasets for training the model doesn’t include “gull”, so that it classified similar candidates instead. You never get results out of the list, and that’s why transfer learning is needed.

## Visualize intermediate layer outputs

Before going to transfer learning, let’s visualize intermediate layer outputs. To show list of layers, run the code below.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# Will allow us to embed images in the notebook
%matplotlib inline

import pandas as pd
pd.DataFrame(model.layers).tail()

We want to visualize outputs of layer 311, `GlobalAveragePooling2D`, so let’s construct a model to output the intermediate layer outputs.

In [ ]:
%%ml_code
from keras.models import Model

# The model which outputs intermediate layer features
intermediate_layer_model = Model(inputs=model.input, 
                                 outputs=model.layers[311].output)

To extract features and visualize, run the following code.

In [ ]:
def extract_features(img_path):
    img = image.load_img(img_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Display image
    display(img)
    
    # Extract features
    features = intermediate_layer_model.predict(x)

    # Visualize
    pd.DataFrame(features.reshape(-1,1)).plot(figsize=(12, 3))
    
extract_features('Indian_elephant.jpeg')
extract_features('Gull.jpeg')

The outputs of `GlobalAveragePooling2D` are 2048 dimensions features. Inception-v3 model classifies 1000 classes by using Dense layer at the end of the network, which uses these features as input. But now, we would like to classify “other” classes. So let’s remove this layer and put another one.

## Add Dense layers for fine tuning

Let’s add dense layers, if we want to classify **two-classes**, the code would be something like this.

In [ ]:
%%ml_code

from keras.layers import Dense

# Connect Dense layers at the end
x = intermediate_layer_model.output
# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# Add a logistic layer -- let's say we have 2 classes
predictions = Dense(2, activation='softmax')(x)

# This is the model we will train which is Transfer Learning model
transfer_model = Model(inputs=intermediate_layer_model.input, outputs=predictions)

## Visualize transfer layer outputs

Let’s visualize transfer layer outputs. To show list of layers, run the code below.

In [ ]:
pd.DataFrame(transfer_model.layers).tail()

At this moment, the model trains all its variables. But we want to train only the dense layers we added, so let’s freeze untrained layers.

In [ ]:
%%ml_code

# Freeze all layers
for layer in transfer_model.layers:
    layer.trainable = False

# Unfreeze last dense layers
transfer_model.layers[312].trainable = True
transfer_model.layers[313].trainable = True

# compile the model (should be done *after* setting layers to non-trainable)
transfer_model.compile(loss='categorical_crossentropy',
                       # Adam is an optimization algorithm that can used instead of 
                       # the classical stochastic gradient descent procedure to update network weights iterative 
                       # based in training data.
                       optimizer='adam',
                       metrics=['accuracy'])

Done! Now we can fine-tune this model for dedicated two-classes classification.

Fine tuning for two-classes classification
Let’s classify the images below. the datasets is named Opera-Capitol datasets, which includes Opera house and Capitol 100 images for each. You can download the code to make this datasets.
https://github.com/hayatoy/deep-learning-datasets

![](https://cdn-images-1.medium.com/max/1600/1*iHxe57geQ_CkXsdYnu-ZWw.png)

## Load dataset

The dataset is compressed as NumPy format and stored in GitHub, you can use it as follows.

In [ ]:
%%ml_code

import requests
from io import BytesIO

url = 'https://github.com/hayatoy/deep-learning-datasets/releases/download/v0.1/tl_opera_capitol.npz'
response = requests.get(url)
dataset = np.load(BytesIO(response.content))

X_dataset = dataset['features']
y_dataset = dataset['labels']

Let’s split the dataset into for train and for test, here I split it 80% for train and 20% for test.

In [ ]:
%%ml_code

from keras.utils import np_utils
from sklearn.model_selection import train_test_split

X_dataset = preprocess_input(X_dataset)
y_dataset = np_utils.to_categorical(y_dataset)
X_train, X_test, y_train, y_test = train_test_split(
    X_dataset, y_dataset, test_size=0.2, random_state=42)

## Fine tuning the model for Opera-Capitol

To train the transfer learning model, just call `fit` function. After that, let’s evaluate the model how it predict correctly.

In [ ]:
%%ml_code

def fine_tuning(epochs=5):
    transfer_model.fit(X_train, y_train, epochs,
                       validation_data=(X_test, y_test))
    loss, acc = transfer_model.evaluate(X_test, y_test)
    print('Loss {}, Accuracy {}'.format(loss, acc))

Invoking the function fine_tuning() below to train the model locally, and this might take you couple of minutes.

In [ ]:
fine_tuning()

In [ ]:
def predict2(img_path):
    img = image.load_img(img_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    # Display image
    display(img)
    
    # Classify image
    preds = transfer_model.predict(x)
    
    # Print predicted classes
    print('Predicted:')
    print("Score {}, Label {}".format(preds[0][0], 'Opera'))
    print("Score {}, Label {}".format(preds[0][1], 'Capitol'))
        
predict2('Opera_house.jpeg')
predict2('United_states_capitol.jpg')

## Cloud ML Engine from Jupyter Notebook

Before you begin, prepare [Google Cloud Platform](https://cloud.google.com/) project, enable billing and install [Google Cloud SDK](https://cloud.google.com/sdk/downloads).

To activate the credentials, enter:

```
% gcloud auth application-default login
```

To list credential account, enter:

```
%gcloud auth list

Credentialed Accounts
ACTIVE  ACCOUNT
*       <myaccount>@<mydomain>
```

To list project, enter:

```
% gcloud config list project

[core]
project = <PROJECT_ID>
```

If it is not, you can set it with this command:

```
% gcloud config set project <PROJECT_ID>
```

In [ ]:
%%ml_code

import tensorflow as tf
from keras import backend as K

def save_model(export_dir):
    # The learning phase flag is a bool tensor (0 = test, 1 = train) to be passed as input to any Keras function 
    # that uses a different behavior at train time and test time.
    K.set_learning_phase(0)
    sess = K.get_session()

    from tensorflow.python.framework import graph_util

    # Make GraphDef of Transfer Model
    g_trans = sess.graph
    # Replaces all the variables in a graph with constants of the same values.
    # If you have a trained graph containing Variable ops, it can be convenient to convert them all to Const ops 
    # holding the same values. This makes it possible to describe the network fully with a single GraphDef file, 
    # and allows the removal of a lot of ops related to loading and saving the variables.
    g_trans_def = graph_util.convert_variables_to_constants(
                    sess,
                    g_trans.as_graph_def(),
                    [transfer_model.output.name.replace(':0','')])

    # Image Converter Model
    with tf.Graph().as_default() as g_input:
        input_b64 = tf.placeholder(shape=(1,), dtype=tf.string, name='input')
        input_bytes = tf.decode_base64(input_b64[0])
        # Detects whether an image is a BMP, GIF, JPEG, or PNG, and performs the appropriate operation to convert 
        # the input bytes string into a Tensor of type uint8.
        image = tf.image.decode_image(input_bytes)
        # Convert image to dtype, scaling its values if needed.
        # Images that are represented using floating point values are expected to have values in the range [0,1). 
        # Image data stored in integer data types are expected to have values in the range [0,MAX], 
        # where MAX is the largest positive representable number for the data type.
        # This op converts between data types, scaling the values appropriately before casting.
        image_f = tf.image.convert_image_dtype(image, dtype=tf.float32)
        # Inserts a dimension of 1 into a tensor's shape.
        input_image = tf.expand_dims(image_f, 0)
        # Return a tensor with the same shape and contents as input.
        output = tf.identity(input_image, name='input_image')

    g_input_def = g_input.as_graph_def()

    with tf.Graph().as_default() as g_combined:
        x = tf.placeholder(tf.string, name="input_b64")

        im, = tf.import_graph_def(g_input_def,
                                  # A dictionary mapping input names (as strings) in graph_def to Tensor objects. 
                                  # The values of the named input tensors in the imported graph will be re-mapped 
                                  # to the respective Tensor values.
                                  input_map={'input:0': x},
                                  return_elements=["input_image:0"])

        pred, = tf.import_graph_def(g_trans_def,
                                     input_map={transfer_model.input.name: im,
                                                 'batch_normalization_1/keras_learning_phase:0': False},
                                     return_elements=[transfer_model.output.name])

        with tf.Session() as sess2:
            inputs = {"inputs": tf.saved_model.utils.build_tensor_info(x)}
            outputs = {"outputs": tf.saved_model.utils.build_tensor_info(pred)}
            signature = tf.saved_model.signature_def_utils.build_signature_def(
                inputs=inputs,
                outputs=outputs,
                method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
            )

            # save as SavedModel
            #export_dir = './savedmodel'
            b = tf.saved_model.builder.SavedModelBuilder(export_dir)
            b.add_meta_graph_and_variables(sess2,
                                           [tf.saved_model.tag_constants.SERVING],
                                           signature_def_map={'serving_default': signature})
            b.save()    

In [ ]:
%%ml_run cloud

# Invoking the function fine_tuning() to train the model.
fine_tuning(epochs=20)

# Remove previous saved model from GS

# Invoking the function save_model() to save the model.
save_model('gs://cloudml_models/transfer_learning/savedmodel')

In [ ]:
# This cell is to prevent "runAll".
# you must wait until ML Engine job finishes
raise Exception('wait until ml engine job finishes..')

Create Model and Version for Online Prediction

In [ ]:
# Options are [europe-west1, us-central1]
# Only need to create models first time.
#!gcloud ml-engine models create OperaCapitol --regions us-central1
!gcloud ml-engine versions create v1 --model OperaCapitol --runtime-version 1.2 --origin gs://$BUCKET/savedmodel

In [ ]:
import os
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery
from googleapiclient import errors

PROJECTID = os.getenv('PROJECTID')
projectID = 'projects/{}'.format(PROJECTID)
modelName = 'OperaCapitol'
modelID = '{}/models/{}'.format(projectID, modelName)

credentials = GoogleCredentials.get_application_default()
ml = discovery.build('ml', 'v1', credentials=credentials)

In [ ]:
import base64
import json

def predict3(img_path):
    # Display image
    img = image.load_img(img_path, target_size=(299, 299))    
    display(img)

    with open(img_path, 'rb') as f:
        b64_x = f.read()

    b64_x = base64.urlsafe_b64encode(b64_x)
    # Keys should be the names of Tensors your deployed model expects as inputs.
    input_instance = dict(inputs=b64_x)
    input_instance = json.loads(json.dumps(input_instance))
    request_body = {"instances": [input_instance]}

    request = ml.projects().predict(name=modelID, body=request_body)
    try:
        response = request.execute()
    except errors.HttpError as err:
        # Something went wrong with the HTTP transaction.
        # To use logging, you need to 'import logging'.
        print('There was an HTTP error during the request:')
        print(err._get_reason())
    print(response)
        
predict3('Opera_house.jpeg')
predict3('United_states_capitol.jpg')